## Cabecera
Montero Barraza Álvaro David

5BV1

Ingenieria en Inteligencia Artificial 


Práctica 5. Analisis de sentimientos

A lo largo de esta practica, utilizaremos diversos modelos para el analisis de sentimientos, utilizando modelos de machine learning, diccionarios y redes neuronales. Ademas de que realizaremos un analisis de una coleccion de reseñas de comida fina en amazon y finalmente compararemos los distintos modelos al ponerlos a prueba con el dataset normalizado y vectorizado segun el modelo que corresponda

In [1]:
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from copy import deepcopy
import re
import random
from collections import Counter
from tensorflow import keras
import tensorflow as tf
import tensorflow_text as tf_text
import numpy as np
import matplotlib.pyplot as plt
import csv
from nltk.corpus import opinion_lexicon
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
nltk.download('opinion_lexicon')
nltk.download('punkt')
from sklearn.model_selection import StratifiedKFold
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
import pysentiment2 as ps
from sklearn.preprocessing import OneHotEncoder
import math


2024-06-26 00:36:27.220982: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-26 00:36:27.599989: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 00:36:28.271543: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /home/alvaromontero/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/alvaromontero/nltk_data...
[nltk_data

## **Funciones**

In [2]:
def randomize(lista):
    randomized = lista[:]  
    random.shuffle(randomized)
    return randomized

# Elimina etiquetas HTML y comentarios dentro del propio texto como indicaciones extra entre parentesis o comentarios entre --
def eliminar_etiquetas(text):
    pattern = re.compile(r'<[^>]+>|\([^)]+\)|--[^\-]+--')
    doc = pattern.sub('', text)
    return doc

# Removemos apostrofes para quedarnos con palabras como cant couldnt o dont
def remover_apostrofes(text):
    pattern = r"\\?'"
    doc = re.sub(pattern, '', text)
    return doc
# Nos quedamos con solo alfabeto eliminando caracteres especiales    
def remover_especiales(text):
    pattern = r"[^a-zA-Z\s.,]"
    doc = re.sub(pattern, ' ', text)
    return doc
# Quitamos dobles espacios para tener texto espaciado solamente por un espacio por palabra
def dobles_espacios(text):
    pattern = r'\s+'
    doc = re.sub(pattern, ' ', text)
    return doc
    
# Removemos stop words unicamente en ingles
def remove_stopwords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

# Lematizacion mediante spacy
nlp = spacy.load("en_core_web_trf")
def lemmatize(doc):
    docs = list(nlp.pipe([doc]))
    lemas = ' '.join([token.lemma_ for doc in docs for token in doc])
    return lemas

def leer_datos(path):
    data = []
    with open(path, 'r', encoding='utf-8') as archivo: 
        reader = csv.reader(archivo)
        header = next(reader, None)  # Obtener la primera línea como encabezado
        for linea in reader:
            data.append(linea)

    return header, data

def put_label(stars):
  if stars<=2:
    return 0
  elif stars==3:
    return 1
  else:
    return 2


/home/alvaromontero/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alvaromontero/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/alvaromontero/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Funciones de vectorización

In [7]:
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm




def vectorizar_tfidf(text):
    # TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(text)
    
    # palabras únicas
    feature_names_tfidf = vectorizer.get_feature_names_out()

    print("Tamaño de Palabras únicas:", len(feature_names_tfidf))
    return tfidf_matrix


def tf_vectorizar_tfidf(texts, batch_size=1000):
    def process_batch(text_batch):
        text_tensor = tf.constant(text_batch)

        # Tokenizador básico
        tokenizer = tf_text.WhitespaceTokenizer()
        tokens = tokenizer.tokenize(text_tensor)

        # Crear un vocabulario
        unique_tokens, _ = tf.unique(tokens.flat_values)

        # Función para calcular frecuencia de documentos (DF)
        def doc_freq(term, tokens):
            return tf.reduce_sum(tf.cast(tf.reduce_any(tf.equal(tokens, term), axis=-1), tf.float32))

        # Calcular DF para cada término
        df = tf.map_fn(lambda term: doc_freq(term, tokens), unique_tokens, dtype=tf.float32)

        # Calcular IDF
        doc_count = tf.cast(len(text_batch), tf.float32)
        idf = tf.math.log(doc_count / (df + 1))

        # Función para calcular TF (frecuencia de término)
        def term_freq(term, tokens):
            return tf.reduce_sum(tf.cast(tf.equal(tokens, term), tf.float32), axis=-1)

        # Calcular TF-IDF
        def compute_tfidf(term):
            tf_value = term_freq(term, tokens)
            idf_value = idf[tf.where(tf.equal(unique_tokens, term))[0][0]]
            return tf_value * idf_value

        tfidf_values = tf.map_fn(lambda term: compute_tfidf(term), unique_tokens, dtype=tf.float32)

        # Crear un tensor TF-IDF por documento
        tfidf_per_doc = tf.stack([tfidf_values[:, i] for i in range(len(text_batch))], axis=1)
        return tfidf_per_doc
    num_texts = len(texts)
    tfidf_results = []
    for i in range(0, num_texts, batch_size):
        text_batch = texts[i:i + batch_size]
        tfidf_batch = process_batch(text_batch)
        tfidf_results.append(tfidf_batch)

    # Combinar los resultados de los lotes
    return tf.concat(tfidf_results, axis=1)

def vectorizar_onehot(input):
    # One-Hot Encoding
    vectorizer = CountVectorizer(binary=True)
    onehot_matrix = vectorizer.fit_transform(input)
    return onehot_matrix

def preprocess_texts(texts):
    processed_texts = []
    doc = list(tqdm(nlp.pipe(texts,disable=['ner','parser']),total=len(texts)))
    for text in doc:
        
        tokens = []
        for token in text:
            # Lematización
            lemma = token.lemma_
            # Stemming
            stem = stemmer.stem(lemma)
            tokens.append(stem)
        processed_texts.append(' '.join(tokens))

    return processed_texts

In [21]:
def prediction(predict_array):
    indice_maximo = predict_array
    if indice_maximo==0:
        print("Sentimiento negativo")
    elif indice_maximo==1:
        print("Sentimiento neutro")
    elif indice_maximo==2:
        print("Sentimiento positivo")

Diccionarios de sentimientos


In [8]:
def hiv4_polarity(text,hiv4):
    tokens = hiv4.tokenize(text)  
    score = hiv4.get_score(tokens)

    polarity = score['Polarity']
    if polarity > 0.33:
        return 2
    elif polarity < -0.33:
        return 0
    else:
        return 1

def analyze_sentiment(text):
    # Tokenizar el texto
    tokens = word_tokenize(text.lower())

    # Obtener palabras positivas y negativas del Opinion Lexicon
    pos_words = set(opinion_lexicon.positive())
    neg_words = set(opinion_lexicon.negative())

    # Calcular el sentimiento
    pos_score = sum(word in pos_words for word in tokens)
    neg_score = sum(word in neg_words for word in tokens)

    # Determinar la orientación general
    if pos_score > neg_score:
        return 1
    elif pos_score < neg_score:
        return 0
    else:
        return 2

## Adquisición de datos

In [9]:
header,raw_dataset=leer_datos("/mnt/d/Descargas/Reviews/Reviews.csv")

## Análisis exploratorio de los datos

In [8]:
print("hay",len(raw_dataset),"datos. Cada uno con",len(header),"caracteristicas")
print("columnas:")
print(header,"\n")
print(raw_dataset[1], "\n")
print(raw_dataset[2])

hay 568454 datos. Cada uno con 10 caracteristicas
columnas:
['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'] 

['2', 'B00813GRG4', 'A1D87F6ZCVE5NK', 'dll pa', '0', '0', '1', '1346976000', 'Not as Advertised', 'Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".'] 

['3', 'B000LQOCH0', 'ABXLMWJIXXAIN', 'Natalia Corres "Natalia Corres"', '1', '1', '4', '1219017600', '"Delight" says it all', 'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion,

In [11]:
columnas = [6,8,9]
reviews = [[row[i] for i in columnas] for row in raw_dataset]
print(reviews[2])#Segmentacion, guardamos score, summary y text

['4', '"Delight" says it all', 'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.']


In [12]:
#Contamos cuantos vectores hay por clase
#Colocamos el label correcto
label_counter = [0,0,0]
for i in range(len(reviews)):
    if reviews[i][0] == '1' or reviews[i][0] == '2':#Mala opinion
        reviews[i][0] = 0
        label_counter[0] += 1
    elif reviews[i][0] == '3':#Opinion regular
        reviews[i][0] = 1
        label_counter[1] += 1
    else:
        reviews[i][0] = 2#Opinion buena
        label_counter[2] += 1
print(label_counter)

[82037, 42640, 443777]


In [13]:
def randomize(lista):
    randomized = lista[:]  
    random.shuffle(randomized)
    return randomized
reviews = randomize(reviews)
print(reviews[0])

[2, 'Addicting Combination of Flavors', "The combination of dark chocolate, nuts, & sea salt make for an addicting combination. I've already made it through two bars since it arrived in the mail earlier today. I also love the fact that it is made with quality ingredients, so even though it approaches dessert like status, it is more healthily than a candy bar and tastes better than many granola bars, as much as I love granola bars. The salt, chocolate and texture reminds me of Ben & Jerry's Chubby Hubby minus the ice cream of course,"]


## Preprocesamiento

In [14]:

#Balance de clases
reviews_compressed = []
label_check = [0,0,0]
for i in range(len(reviews)):
    if label_check[0] == label_check[1] == label_check[2] == 42640:
        break
    if reviews[i][0] == 0 and label_check[0] < 42640:
        reviews_compressed.append(reviews[i])
        label_check[0] += 1
    elif reviews[i][0] == 1 and label_check[1] < 42640:
        reviews_compressed.append(reviews[i])
        label_check[1] += 1
    elif reviews[i][0] == 2 and label_check[2] < 42640:
        reviews_compressed.append(reviews[i])
        label_check[2] += 1
print(label_check)
print(len(reviews_compressed))
reviews = reviews_compressed

[42640, 42640, 42640]
127920


In [22]:
#Prescindimos de todo menos el score y el summary
X=[str(text[1]) for text in reviews]
y=[label[0] for label in reviews]
print(X[0])
prediction(y[0])


Addicting Combination of Flavors
Sentimiento positivo


In [23]:
#Procesamiento del texto
nlp=spacy.load('en_core_web_trf')
X=[text.lower() for text in X]
X=[eliminar_etiquetas(text) for text in X]
X=[remover_apostrofes(text) for text in X]
X=[remover_especiales(text) for text in X]
X=[dobles_espacios(text) for text in X]

X_wS=deepcopy(X)

X=[remove_stopwords(text) for text in X]
X_WoS=deepcopy(X)


In [24]:
for i in range(15):
    print(X_WoS[i],end=' ')
    prediction(y[i])

addicting combination flavors Sentimiento positivo
gets dog eat Sentimiento positivo
product china dont buy Sentimiento negativo
bad Sentimiento neutro
best idea ever Sentimiento positivo
burst refreshment Sentimiento positivo
great stuff Sentimiento positivo
premium dog food moderate price Sentimiento positivo
great cereal Sentimiento positivo
excellent brew Sentimiento positivo
great coconut oil Sentimiento positivo
awesome Sentimiento positivo
roasted salty Sentimiento positivo
great smoothies kids love Sentimiento positivo
numi chocolate tea bags Sentimiento negativo


In [14]:
#Tokenizacion de reseñas
X_wS=preprocess_texts(X_wS)       

100%|██████████| 127920/127920 [11:05<00:00, 192.33it/s]


In [25]:
X_WoS=preprocess_texts(X_WoS)#Texto sin stopwords

100%|██████████| 127920/127920 [08:38<00:00, 246.93it/s]


In [26]:
print(len(X_WoS))
print(X_WoS[10])
tfids_w=vectorizar_tfidf(X_WoS)#Vectorización a texto sin stopwords
one_hot_w=vectorizar_onehot(X_WoS)

127920
great coconut oil
Tamaño de Palabras únicas: 12569


In [19]:
print(tfids_w[0])

  (0, 10630)	0.33837600037899407
  (0, 2401)	0.4079830380935052
  (0, 4016)	0.5090840035370278
  (0, 7545)	0.39789795831926666
  (0, 3320)	0.5491468065131392


## Análsis de sentimientos con diccionarios

In [27]:
text_analysis = []
hiv4 = ps.HIV4()

for i in range(len(X_WoS)):
    if i % 12000 == 0:
        print("Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)")
    harvard_pol_text = hiv4_polarity(X_WoS[i],hiv4)
    nltk_pol_text = analyze_sentiment(X_WoS[i])
    text_analysis.append([harvard_pol_text,nltk_pol_text])

Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)


In [28]:
print(text_analysis[:10])   

[[0, 0], [0, 2], [2, 2], [0, 0], [1, 1], [1, 1], [2, 1], [2, 2], [2, 1], [2, 1]]


In [29]:
accuracy_dict = [0,0]
for i in range(len(text_analysis)):
    if y[i] == text_analysis[i][0]:
        accuracy_dict[0] += 1
    if y[i] == text_analysis[i][1]:
        accuracy_dict[1] += 1
        
accuracy_dict[0] = accuracy_dict[0] / 127920
accuracy_dict[1] = accuracy_dict[1] / 127920

In [31]:
print("Precision usando diccionarios")
print("HIV4 ------------------ Opinion Lexicon")
print(accuracy_dict)

Precision usando diccionarios
HIV4 ------------------ Opinion Lexicon
[0.4104596622889306, 0.33610068792995623]


## Análisis de sentimientos con Machine Learning



In [12]:
tfidf_array=vectorizar_tfidf(X_WoS)

Tamaño de Palabras únicas: 16889


In [13]:
from sklearn.linear_model import LogisticRegression
logistic_reg = LogisticRegression(max_iter=1000)
decision_tree = DecisionTreeClassifier()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [77]:
#Regresion logistica
scores1 = cross_val_score(logistic_reg, tfidf_array, y, cv=cv)

print("Accuracy for each fold:", scores1)
print("Mean accuracy:", scores1.mean())

Accuracy for each fold: [0.4793621  0.48397436 0.48217636 0.48182458 0.48542058]
Mean accuracy: 0.4825515947467167


In [78]:
scores2 = cross_val_score(decision_tree, tfidf_array, y, cv=cv)

print("Accuracy for each fold:", scores2)
print("Mean accuracy:", scores2.mean())

Accuracy for each fold: [0.47005941 0.47170106 0.47088024 0.47416354 0.47494528]
Mean accuracy: 0.4723499061913696


In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
svc = SVC()

# Definir los hiperparámetros para la búsqueda en cuadrícula
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001]}

# Configurar GridSearchCV con n_jobs=-1 para usar todos los núcleos disponibles
grid_search = GridSearchCV(svc, param_grid, n_jobs=16, cv=5)
scores = cross_val_score(svc, tfidf_array, y, cv=cv,n_jobs=-1)

# Imprimir los resultados de la cross-validation
print("Accuracy for each fold:", scores)
print("Mean accuracy:", scores.mean())

Accuracy for each fold: [0.72568793 0.72291276 0.72424171 0.72267824 0.72127111]
Mean accuracy: 0.723358348968105


## Análisis de sentimientos usando word embbedings y redes neuronales

In [16]:
label_encoder=LabelEncoder()
label_dict1=label_encoder.fit_transform(y)
encoded_y=np.array(y)
encoded_y = tf.keras.utils.to_categorical(encoded_y, 3)
print(encoded_y[123])

[0. 0. 1.]


Capa con embeddings preconstruidos

In [15]:
embeddings_dict = {}
with open("/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/glove.6B/glove.6B.100d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector
print(f"{len(embeddings_dict)} vectores almacenados")

400000 vectores almacenados


In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

t = Tokenizer()
t.fit_on_texts(X_WoS)
vocab_size = len(t.word_index) + 1

encoded_docs = t.texts_to_sequences(X_WoS)

# Definimos el maximo size de cada documento
max_length = 150
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [18]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
 embedding_vector = embeddings_dict.get(word)
 if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector


In [2]:

from tensorflow.keras.callbacks import EarlyStopping
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
acc_scores = []

iteracion = 0
# Iteracion del modelo por KFolds
for train_index, test_index in kf.split(padded_docs, y):
    iteracion += 1
    print("[K-Fold] Iteracion: ", iteracion)

    # Definición del modelo
    model = tf.keras.models.Sequential()
    e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)
    model.add(e)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Dividir los datos en entrenamiento y prueba
    X_train, X_test = padded_docs[train_index], padded_docs[test_index]
    y_train, y_test = encoded_y[train_index], encoded_y[test_index]

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Entrenamiento del modelo
    model.fit(X_train, y_train, epochs=10, validation_split=0.1, callbacks=[early_stopping], verbose=1)

    # Evaluación del modelo
    loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
    acc_scores.append(accuracy)
    print(f"Iteracion {iteracion} - Precisión: {accuracy:.4f}")

print("Precisión promedio: {:.2f}%".format(np.mean(acc_scores) * 100))

[K-Fold] Iteracion:  1
0.5978749990463257
[K-Fold] Iteracion:  2
0.5911666750907898
[K-Fold] Iteracion:  3
0.5985000133514404
[K-Fold] Iteracion:  4
0.5980833172798157
[K-Fold] Iteracion:  5
0.5957083106040955
Precision promedio: 59.63%
      


Capa con embeddings aprendidos del texto

In [60]:
vocab_size = 50000
# integer encode the documents
encoded_docs = [vectorizar_onehot(d, vocab_size) for d in X_WoS]
#print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [3]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
acc_scores = []

iteracion = 0

# Iteración del modelo por KFolds
for train_index, test_index in kf.split(padded_docs, y):
    iteracion += 1
    print("[K-Fold] Iteracion: ", iteracion)

    model = tf.keras.layers.Sequential()

    model.add(tf.keras.layers.Embedding(vocab_size, 8, input_length=max_length))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    X_train, X_test = padded_docs[train_index], padded_docs[test_index]
    y_train, y_test = encoded_y[train_index], encoded_y[test_index]

    model.fit(X_train, y_train, epochs=50, verbose=0)
    
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    acc_scores.append(accuracy)
    print(accuracy)

print("Precisión promedio: {:.2f}%".format(np.mean(acc_scores) * 100)) 


[K-Fold] Iteracion:  1
0.6744583249092102
[K-Fold] Iteracion:  2
0.6714583039283752
[K-Fold] Iteracion:  3
0.6805833578109741
[K-Fold] Iteracion:  4
0.6728749871253967
[K-Fold] Iteracion:  5
0.6694999933242798
Precisión promedio: 67.38%
